In [72]:
!pip install mysql-connector-python
!pip install pymongo

  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Can't uninstall 'protobuf'. No files were found to uninstall.
  ERROR: Can't roll back protobuf; was not uninstalled
ERROR: Could not install packages due to an OSError: [Errno 30] Read-only file system: '/nix/store/j53722mnyjm51kdnqmhgn10ddns350n1-python3-3.12.11-env/lib/python3.12/site-packages/protobuf-3.20.3-nspkg.pth'



## MySQL

In [70]:
import mysql.connector
from mysql.connector import errorcode

DB_NAME = "Assn3"
TABLE_NAME = "Wiki_Edit"

TABLES = {}
TABLES['Wiki_Edit'] = (
    "CREATE TABLE `Wiki_Edit` ("
    "  `RevisionID` int NOT NULL,"
    "  `ArticleName` varchar(500) NOT NULL,"
    "  `EditDate` date NOT NULL,"
    "  `UserName` varchar(50), "
    "  PRIMARY KEY (`RevisionID`)"
    ")")

cnx = mysql.connector.connect(
  host="localhost",
  user='root'
)

cursor = cnx.cursor()

def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        cnx.database = DB_NAME
    else:
        print(err)

for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

add_edit = ("INSERT INTO Wiki_Edit "
              "(RevisionID, ArticleName, EditDate, UserName) "
              "VALUES (%(RevisionID)s, %(ArticleName)s, %(EditDate)s, %(UserName)s)")

cursor.execute("SELECT COUNT(*) FROM Wiki_Edit;")
count = cursor.fetchone()[0]
if count == 0:
    with open("wiki_edit.txt") as file:
        lines = file.readlines()
        for line in lines:
            values = line.split(" ")
            id, article, date, username = values
            edit = {"RevisionID": id, "ArticleName": article, "EditDate": date, "UserName": username}
            cursor.execute(add_edit, edit)
    cnx.commit()

cursor.execute("SELECT ArticleName, COUNT(*) AS EDITS FROM Wiki_Edit GROUP BY ArticleName ORDER BY Edits DESC LIMIT 1;")

most_edited = cursor.fetchone()
print(f"The most edited article is {most_edited[0]} with {most_edited[1]} edits.")

    
cursor.close()
cnx.close()

Creating table Wiki_Edit: already exists.
The most edited article is 2004_Indian_Ocean_earthquake with 186 edits.


## NoSQL

In [71]:
from typing import TypedDict 
from pymongo import MongoClient
from pymongo.database import Database
import json


DB_NAME = "assn3"

class User(TypedDict):
    id: int
    name: str
    screen_name: str
    location: str
    description: str
    url: str
    followers_count: int
    friends_count: int
    favourites_count: int
    statuses_count: int
    created_at: str
    verified: bool
    follower: bool
    

def populate_db(database):
    collections = ["followers", "followees"]
    for name in collections:
        collection = database[name]
        with open(f'{name}.json', 'r') as file:
            data = json.load(file)
            for entry in data:
                collection.insert_one(User(
                    name=entry["name"],
                    screen_name=entry["screen_name"],
                    location=entry["location"],
                    description=entry["description"],
                    url=entry["url"],
                    followers_count=entry["followers_count"],
                    friends_count=entry["friends_count"],
                    favourites_count=entry["favourites_count"],
                    statuses_count=entry["statuses_count"],
                    created_at=entry["created_at"],
                    verified=entry["verified"],
                    follower = name == "follower"
                ))


uri = "mongodb://localhost:27017"
client = MongoClient(uri)


def average_followers(collection):
    results = collection.aggregate([{
        "$group": {
         "_id": None,
         "avg_followers": { "$avg": "$followers_count" }
        }
    }])
    return next(results)["avg_followers"]
    

def number_of_verified(collection):
    result = collection.count_documents({
        "verified": True
    })
    return result

def average_favorites_count(collection):
    results = collection.aggregate([{
        "$group": {
         "_id": None,
         "avg_fav_count": { "$avg": "$favourites_count" }
        }
    }])
    return next(results)["avg_fav_count"]
    
def average_favorites_count_comb(database):
    results = database["followers"].aggregate([
        {
        "$unionWith": "followees"
        },
        {
        "$group": {
         "_id": None,
         "avg_fav_count": { "$avg": "$favourites_count" }
        }
    }])
    return next(results)["avg_fav_count"]

    
try:
    database = client[DB_NAME]
    populate_db(database)
    avg_folr = average_followers(database["followers"])
    print(f"Average followers of followers: {avg_folr}")
    avg_fole = average_followers(database["followees"])
    print(f"Average followers of followees: {avg_fole}")
    num_v_followers = number_of_verified(database["followers"])
    print(f"Number of verified followers: {num_v_followers}")
    num_v_followees = number_of_verified(database["followees"])
    print(f"Number of verified followees: {num_v_followees}")
    print(f"Number of verified combined: {num_v_followees + num_v_followers}")
    avg_fav_folr = average_favorites_count(database["followers"])
    print(f"Average favorite count of followers: {avg_fav_folr}")
    avg_fav_fole = average_favorites_count(database["followees"])
    print(f"Average favorite count of followees: {avg_fav_fole}")
    avg_comb = average_favorites_count_comb(database)
    print(f"Average favorites count of followers and followees combined: {avg_comb}")


finally:
    client.close()

Average followers of followers: 2983.2087912087914
Average followers of followees: 1075218.8629032257
Number of verified followers: 43
Number of verified followees: 473
Number of verified combined: 516
Average favorite count of followers: 1899.6813186813188
Average favorite count of followees: 6092.4112903225805
Average favorites count of followers and followees combined: 4317.813953488372
